In [7]:
# Load pre-trained model and evaluate it on the test set
import os
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

from torchvision import datasets

# import datetime

from datetime import datetime



In [12]:
if torch.backends.mps.is_available():
	device = torch.device("mps")
	x = torch.ones(1, device=device)
	print(x)

elif torch.backends.cuda.is_built():
	device = torch.device("cuda")
	x = torch.ones(1, device=device)
	print (x)
else:
	device = None
	print ("MPS device not found.")

tensor([1.], device='mps:0')


In [27]:
# Load the model
model_epoch = 18
model_path = f"models/model-20240225-180018/epoch-{model_epoch}/model-{model_epoch}-complete"

reference_model = torch.load(model_path)

In [28]:
# Define transformations
transformations = transforms.Compose(
    [
        # TODO: Resize to 256 x 480
        # TODO: Scale Augmentation
        transforms.Resize((224, 224)),  # Crop to 224 x 224 ?
        transforms.ToTensor(),  # Convert to Tensor before numeric manipulation (normalisation)
        # TODO: Mean Substraction ?
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        # Normalise to [0, 1] ?
        # TODO: Color Augmentation
    ]
)

In [29]:
def test_predict():
    reference_model.eval()  # Set model to evaluation mode

    test_dataset = datasets.ImageFolder(
        'data/test',
        transform=transformations
    )
    test_dataset_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=64, shuffle=False, num_workers=4
    )

    counter = 0
    sample_idx = 0

    current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    with open(f'submit-{current_time}.csv', 'w') as f:
        f.write('ID,prediction\n')
        with torch.no_grad():
            for data in test_dataset_loader:

                inputs, labels = data

                # img_0 = inputs[0]
                # plt.imshow(img_0.permute(1, 2, 0))
                # plt.axis('off')
                # plt.show();

                # plt.imshow(img.permute(1, 2, 0))
                # plt.show();

                inputs, labels = inputs.to(device), labels.to(device)


                outputs = reference_model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                for label in predicted:
                    # file_name = test_dataset.samples[label][0].split('/')[-1]
                    file_name = test_dataset.samples[sample_idx][0].split('/')[-1]
                    sample_idx += 1
                    # remove .JPEG
                    file_name = file_name[:-5]
                    f.write(f'{file_name},{label.item()}\n')
                    # counter += 1
                    # if counter == 10:
                    #     break
                # if counter == 10:
                #     break
                # print(predicted)
                # for label in predicted:
                #     print(label.item())
                #     print(f'Prediction: {DOG_BREEDS[label.item()][0] + 1}. {DOG_BREEDS[label.item()][1]} ({DOG_BREEDS[label.item()][2]})')
                #     if counter == 10:
                #         break
                #     counter += 1
                # if counter == 10:
                #     break


In [30]:
test_predict()